Загрузка датасетов

In [1]:
import pandas as pd
import numpy as np

In [2]:
df_ratings = pd.read_csv('drive/MyDrive/dataset/ratings.csv')
df_ratings['date'] = pd.to_datetime(df_ratings['timestamp'],unit='s')
df_ratings = df_ratings[['date','userId','movieId','rating']]
df_ratings.columns = ['date','userid','movieid','rating']

In [3]:
df_ratings.head()

,date,userid,movieid,rating
0,2009-12-14 02:52:24,1,31,2.5
1,2009-12-14 02:52:59,1,1029,3.0
2,2009-12-14 02:53:02,1,1061,3.0
3,2009-12-14 02:53:05,1,1129,2.0
4,2009-12-14 02:53:25,1,1172,4.0


In [4]:
usecols = ['id','title']
df_movies = pd.read_csv('drive/MyDrive/dataset/movies.csv',usecols=usecols)
df_movies.id = pd.to_numeric(df_movies.id , errors='coerce').astype('Int32')
df_movies = df_movies.dropna()

In [5]:
df_movies.head()

,id,title
0,862,Toy Story
1,8844,Jumanji
2,15602,Grumpier Old Men
3,31357,Waiting to Exhale
4,11862,Father of the Bride Part II


Реализация рекомендательной системы на основе подбора элементов

In [20]:
df = pd.merge(df_ratings,df_movies,how='left',left_on='movieid', right_on='id')
df = df[~df.id.isnull()]
df = df.loc[:,['userid','movieid','rating','title']]

In [37]:
print(df.head(10))
print('-----------------')
print(df.shape)
print('-----------------')
print(df.isnull().sum())

    userid  movieid  rating                           title
10       1     1371     2.5                       Rocky III
11       1     1405     1.0                           Greed
13       1     2105     4.0                    American Pie
15       1     2193     2.0                        My Tutor
16       1     2294     2.0  Jay and Silent Bob Strike Back
17       1     2455     2.5            Confidentially Yours
21       2       17     5.0                        The Dark
26       2       62     3.0           2001: A Space Odyssey
27       2      110     4.0               Three Colors: Red
28       2      144     3.0                 Wings of Desire
-----------------
(44994, 4)
-----------------
userid     0
movieid    0
rating     0
title      0
dtype: int64


In [34]:
movie_ratings = df.pivot_table(index=['userid'],columns=['title'],values='rating')

In [39]:
movie_ratings_ = movie_ratings ['Rocky III']

In [40]:
similar_movies = movie_ratings.corrwith(movie_ratings_)

/usr/local/lib/python3.7/dist-packages/numpy/lib/function_base.py:2551: RuntimeWarning: Degrees of freedom <= 0 for slice
  c = cov(x, y, rowvar)
/usr/local/lib/python3.7/dist-packages/numpy/lib/function_base.py:2480: RuntimeWarning: divide by zero encountered in true_divide
  c *= np.true_divide(1, fact)


In [41]:
similar_movies = similar_movies.dropna()

In [55]:
df_similar = pd.DataFrame(similar_movies)
df_similar.columns=['similar']

In [56]:
df_similar.head()

,similar
title,
10 Items or Less,-1.000000
"10,000 BC",1.000000
1984,1.000000
"20,000 Leagues Under the Sea",0.432122
2001: A Space Odyssey,0.063900


In [44]:
ratings_count = 100

In [46]:
movie_stats = df.groupby('title').agg({'rating': [np.size, np.mean]})

In [47]:
popular_movies = movie_stats['rating']['size'] >= ratings_count

In [59]:
movie_stats[popular_movies]. \
                            sort_values([('rating', 'mean')], ascending=False). \
                            join(df_similar,on='title').sort_values([('similar')], ascending=False)[:10]

/usr/local/lib/python3.7/dist-packages/pandas/core/reshape/merge.py:643: UserWarning: merging between different levels can give an unintended result (2 levels on the left,1 on the right)
  warnings.warn(msg, UserWarning)


,"(rating, size)","(rating, mean)",similar
title,,,
Confession of a Child of the Century,121.0,4.235537,0.790569
Live and Let Die,125.0,3.396000,0.611856
Rope,190.0,3.663158,0.550350
Romeo + Juliet,110.0,3.472727,0.545838
Fools Rush In,117.0,3.982906,0.543792
Tough Enough,106.0,3.084906,0.532055
The Conversation,196.0,3.397959,0.528508
A Clockwork Orange,102.0,3.102941,0.520547
The Bourne Supremacy,106.0,3.915094,0.511744
